In [10]:
class data:    
    
    import time 
    import selenium 
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import requests
    import os
    from tensorflow.keras.utils import to_categorical
    
    DRIVER_PATH = '/home/trey/Sharpest-Minds-Project/chromedriver'
    wd = webdriver.Chrome(DRIVER_PATH)
    search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"

    def __init__(self,search_name):
        wd.get(search_url.format(q=search_name))
        
    def scrape_and_save(search_name,web_driver = wd):
        body = wd.find_element_by_tag_name("body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2.5)
        main = wd.find_elements_by_class_name("rg_i.Q4LuWd")
        wd.quit()
        links = [main[i].get_attribute('src') for i in range(len(main))]
        images = []
        for image in links:
    # making sure type of data is an image containing the string google encodes for an image
            if type(image) == str:
        # checking conditional text so Image library can open based on based64 conversion
                if image[0:4] == 'data':
            # replace text so we can open it
                    new = image.replace("data:image/jpeg;base64,","")
            # adding equals at the end for decoding
                    if new[-2:] != '==':
                        new_edit = new + '=='
                # image becomes Image object
                        new_image = (Image.open(BytesIO(base64.b64decode(new_edit)))).resize((150,150))
                        images.append(new_image)
                    else:
                        new_image = (Image.open(BytesIO(base64.b64decode(new)))).resize((150,150))
                        images.append(new_image)
                if image[0:4] == 'http':
                    new = requests.get(image)
                    new_image = Image.open(BytesIO(image.content))
                    images.append(new_image)
# creating directories, saving images, and creating names for images as jpeg files
        os.mkdir('Images')
        os.mkdir('Images/training')
        os.mkdir('Images/testing')
        index = 0 
        for i in images:
            #  images = original_lst = list(range(10))
            # i = images[:int(len(images*0.8))]
            new_lst = original_lst[:int(len(original_lst)*0.9)]
            print(new_lst)
            i.save('Images/training'+'/'+string(index)+'.jpeg')
            index += 1 
# add channels to image 


ModuleNotFoundError: No module named 'tensorflow'

In [15]:
search_name = 'dogs'

In [16]:
import time 
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import os

    
DRIVER_PATH = '/home/trey/Sharpest-Minds-Project/chromedriver'
wd = webdriver.Chrome(DRIVER_PATH)
search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"

#def __init__(self,search_name):
wd.get(search_url.format(q=search_name))
        
#def scrape_and_save(search_name,web_driver = wd):
body = wd.find_element_by_tag_name("body")
body.send_keys(Keys.PAGE_DOWN)
time.sleep(2.5)
main = wd.find_elements_by_class_name("rg_i.Q4LuWd")
wd.quit()
links = [main[i].get_attribute('src') for i in range(len(main))]
images = []
for image in links:
# making sure type of data is an image containing the string google encodes for an image
    if type(image) == str:
        # checking conditional text so Image library can open based on based64 conversion
        if image[0:4] == 'data':
            # replace text so we can open it
            new = image.replace("data:image/jpeg;base64,","")
            # adding equals at the end for decoding
            if new[-2:] != '==':
                new_edit = new + '=='
                # image becomes Image object
                new_image = (Image.open(BytesIO(base64.b64decode(new_edit)))).resize((150,150))
                images.append(new_image)
            else:
                new_image = (Image.open(BytesIO(base64.b64decode(new)))).resize((150,150))
                images.append(new_image)
        if image[0:4] == 'http':
            new = requests.get(image)
            new_image = Image.open(BytesIO(image.content))
            images.append(new_image)
# creating directories, saving images, and creating names for images as jpeg files
os.mkdir('Images')
os.mkdir('Images/training')
os.mkdir('Images/testing')
index = 0 
for i in images:
    #  images = original_lst = list(range(10))
        # i = images[:int(len(images*0.8))]
        #new_lst = original_lst[:int(len(original_lst)*0.9)]
        #print(new_lst)
    i.save('Images/training'+'/'+string(index)+'.jpeg')
    index += 1 

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59383): Max retries exceeded with url: /session/b3cdb1a8ef2aae2d0e4bcdd41fb72faf/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3453cae310>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [13]:
scape_and_save('dogs')

NameError: name 'scape_and_save' is not defined